In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
loader = PyPDFLoader("C:\\Users\\yarra\\Downloads\\EBS Developer\\EBS Developer\\Kiran Reddy A_EBS Developer.pdf")
docs = loader.load()
docs

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-06-19T13:48:04+05:30', 'author': 'kiran kumar reddy avuthu', 'moddate': '2025-06-19T13:48:04+05:30', 'source': 'C:\\Users\\yarra\\Downloads\\EBS Developer\\EBS Developer\\Kiran Reddy A_EBS Developer.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Page: 1  \nKiran Kumar Reddy Avuthu \nOracle EBS Technical Lead Consultant \nE-mail: mailtoavuthu@gmail.com \nPhone: +91 8095689689  \nProfessional Summary \n \n● Having 14+ Years of IT experience in Oracle Applications 11i/R12 Development, supporting, Enhancements and \nUpgrade projects. \n● Have Experience in Building Custom Applications using Oracle ERP. \n● Have Experience in Managing Client and Managing Offshore Team. \n● Have Onsite Experience. Worked in UK for one year to Gather requirement and delivery the Product as per the \nclient requirement. \n● Having good knowledge on SCM

In [3]:
from ollama import Client

In [4]:
client = Client(host='http://localhost:11434')

In [5]:
# Step 3: Strict prompt to enforce list output
prompt = f"""
Extract only the **technical skills** from the following resume text. 

✅ STRICTLY RETURN:
- A **valid Python list**
- Do NOT include explanations, numbering, or categories
- Only the skills (e.g., tools, programming languages, platforms, APIs, frameworks)

Resume:
\"\"\"
{docs}
\"\"\"
"""

# Step 4: Query the Ollama 7B model
response = client.chat(
    model='llama2:7b',  # Replace with your actual 7B model name if different
    messages=[{"role": "user", "content": prompt}]
)

# Step 5: Print the clean list
tech_skills = response['message']['content']
print("🛠️ Extracted Technical Skills:")
print(tech_skills)

🛠️ Extracted Technical Skills:
The following are the technical skills mentioned in the resume:

1. Oracle Apps
2. PL/SQL
3. SQL*Loader
4. TOAD
5. POSTMAN
6. REST API/SOAP
7. Oracle Reports
8. Unix/Linux
9. Agile model
10. JIRA
11. PERSONA (a tool for managing and tracking issues)
12. Oracle Developer
13. Microsoft Office

Note that some of these skills may be mentioned multiple times in the resume, but I have listed them here only once for clarity.


In [6]:
import re
# Step 1: Extract skill lines
skills = re.findall(r'\d+\.\s+(.*)', tech_skills)

# Step 2: Clean and expand sub-skills
expanded_skills = []
for skill in skills:
    if '(' in skill and ')' in skill:
        base, extras = re.match(r'^(.*?)\s*\((.*?)\)', skill).groups()
        expanded_skills.append(base.strip())
        # Remove phrases like 'such as', 'like' from extras
        extras_cleaned = re.sub(r'(?i)\b(such as|like)\b', '', extras)
        extra_items = [e.strip() for e in extras_cleaned.split(',')]
        expanded_skills.extend(extra_items)
    else:
        expanded_skills.append(skill.strip())

# Step 3: Final cleanup - remove empty strings
final_skills = [s for s in expanded_skills if s]

# Output
print("✅ Cleaned Technical Skills List:")
print(final_skills)

✅ Cleaned Technical Skills List:
['Oracle Apps', 'PL/SQL', 'SQL*Loader', 'TOAD', 'POSTMAN', 'REST API/SOAP', 'Oracle Reports', 'Unix/Linux', 'Agile model', 'JIRA', 'PERSONA', 'a tool for managing and tracking issues', 'Oracle Developer', 'Microsoft Office']


In [7]:
prompt = f"""
From the following resume text, extract only the **total number of years and months of professional work experience** the candidate has. 

Just return one short sentence like:
"Total experience: X years Y months"

Do not include any other information.

Resume:
\"\"\"
{docs}
\"\"\"
"""

In [8]:
response = client.chat(
    model='llama2:7b',  # or 'llama2:7b', 'mistral', etc.
    messages=[{"role": "user", "content": prompt}]
)

In [9]:
experience_count = response['message']['content']
print("📊 Total Work Experience:")
print(experience_count)

📊 Total Work Experience:
Total experience: 14 years

Experience in Oracle Applications development, support, enhancements, and upgrades for various clients including Assurant Solutions, Carphone UK, and Registry Trust. Worked on multiple projects involving customizations, reports, interfaces, and SQL*Loader control programs. Developed value sets, concurrent programs, and reports in Oracle SCM, INV, PO, AP, AR, GL modules.


# Filteration

In [10]:
def extract_years_from_string(exp_str):
    match = re.search(r'(\d+)(?:\.\d+)?\s*years?', exp_str.lower())
    if match:
        return int(match.group(1))
    return 0
resume_experience_string = experience_count
resume_experience = extract_years_from_string(resume_experience_string)
print(resume_experience)

14


In [11]:
# ✅ Inputs
mandatory_input = input("✅ Enter mandatory skills (comma-separated): ").strip()
good_input = input("✅ Enter good to have skills (comma-separated): ").strip()
min_experience_required = int(input("✅ Enter experience required: "))  # in years

mandatory_skills = [s.strip() for s in mandatory_input.split(",")]
good_to_have_skills = [s.strip() for s in good_input.split(",")]

# ✅ Extracted from resume (example)
resume_skills = final_skills
#print(type(resume_skills))

resume_experience = resume_experience # in years


# ✅ Evaluate Function
def evaluate_candidate(resume_skills, resume_exp, mandatory, good, min_exp):
    resume_skills_lower = [skill.lower() for skill in resume_skills]

    # Match skills
    matched_mandatory = [s for s in mandatory if s.lower() in resume_skills_lower]
    missing_mandatory = [s for s in mandatory if s.lower() not in resume_skills_lower]
    matched_good = [s for s in good if s.lower() in resume_skills_lower]
    missing_good = [s for s in good if s.lower() not in resume_skills_lower]

    # Match experience
    experience_matched = resume_exp >= min_exp

    # Scores
    mandatory_score = len(matched_mandatory) / len(mandatory) * 100 if mandatory else 100
    good_score = len(matched_good) / len(good) * 100 if good else 0
    experience_score = 100 if experience_matched else (resume_exp / min_exp) * 100
    fit_score = round((mandatory_score * 0.6 + good_score * 0.2 + experience_score * 0.2), 2)

    # Fit Status
    is_fit = not missing_mandatory and experience_matched
    reason = "Candidate is a good fit." if is_fit else "Candidate is not a good fit due to:"
    if not is_fit:
        if missing_mandatory:
            reason += f" missing mandatory skills: {missing_mandatory}."
        if not experience_matched:
            reason += f" insufficient experience ({resume_exp} < {min_exp} years)."

    return {
        "✅ Matched Mandatory Skills": matched_mandatory,
        "❌ Missing Mandatory Skills": missing_mandatory,
        "✅ Matched Good-to-Have Skills": matched_good,
        "❌ Missing Good-to-Have Skills": missing_good,
        "📊 Resume Experience (years)": resume_exp,
        "📌 Experience Matched": "Yes" if experience_matched else "No",
        "🎯 Fit Score (%)": fit_score,
        "📌 Fit Status": "Fit" if is_fit else "Not Fit",
        "📝 Reason": reason
    }


# ✅ Run Evaluation
result = evaluate_candidate(
    resume_skills=resume_skills,
    resume_exp=resume_experience,
    mandatory=mandatory_skills,
    good=good_to_have_skills,
    min_exp=min_experience_required
)

# ✅ Print Results
for key, value in result.items():
    print(f"{key}: {value}")

✅ Enter mandatory skills (comma-separated):  Oracle Apps, Oracle EBS, PL/SQL, SQL*Loader, TOAD, POSTMAN, REST API
✅ Enter good to have skills (comma-separated):  Python, .Net, Microsoft Office
✅ Enter experience required:  5


✅ Matched Mandatory Skills: ['Oracle Apps', 'PL/SQL', 'SQL*Loader', 'TOAD', 'POSTMAN']
❌ Missing Mandatory Skills: ['Oracle EBS', 'REST API']
✅ Matched Good-to-Have Skills: ['Microsoft Office']
❌ Missing Good-to-Have Skills: ['Python', '.Net']
📊 Resume Experience (years): 14
📌 Experience Matched: Yes
🎯 Fit Score (%): 69.52
📌 Fit Status: Not Fit
📝 Reason: Candidate is not a good fit due to: missing mandatory skills: ['Oracle EBS', 'REST API'].
